Assume that our data frame has the NAME of the runner and the TIME they took in their previous races, ordered by DATE. Our logic is to get the rolling EWM and then to keep the last for each runner

In [28]:
## generate random date
import datetime
import random
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2021, 2, 1)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
random_date = start_date + datetime.timedelta(days=random_number_of_days)
print(random_date)                                             

2020-06-16


datetime.date(2020, 4, 8)

In [15]:

import pandas as pd
import numpy as np
## convert it to data
#df['DATE'] = pd.to_datetime(df.DATE)
## sort by date
#df.sort_values('DATE', inplace=True)
#df['mean_tmp']=df.groupby('NAME')['TIME'].transform(lambda x: x.ewm(alpha=0.30).mean())
#df['std_tmp']=df.groupby('NAME')['TIME'].transform(lambda x: x.ewm(alpha=0.30).std())
## remove the NAN in Std
#df.dropna(subset=['std_tmp'], inplace=True)
## get the most recent observation of the EWM
#runner= df.groupby('NAME')[['mean_tmp', 'std_tmp']].last()
#runner.reset_index(inplace=True)
#runner.columns = ['NAME', 'mean', 'std']
#runner


## A, 13.1, 12-01-2020
## A, 13.2, 12-10-2020
## 

Assume that we come up with the following mean and standard deviation for the 8 runners.

In [16]:
runner = pd.DataFrame({'NAME':["A","B","C","D","E","F","G","H"],
                       'mean': [13.11, 13.17, 12.99, 12.96, 13.25, 13.00, 13.40, 13.29],
                       'std': [0.15, 0.15, 0.17, 0.20, 0.14, 0.16, 0.17, 0.2]})
runner

,NAME,mean,std
0,A,13.11,0.15
1,B,13.17,0.15
2,C,12.99,0.17
3,D,12.96,0.20
4,E,13.25,0.14
5,F,13.00,0.16
6,G,13.40,0.17
7,H,13.29,0.20


In [17]:
# number of simulations
np.random.seed(5)
# number of simulations
sims = 1000
runner['monte_carlo'] = runner.apply(lambda x:np.random.normal(x['mean'], x['std'], sims), axis=1)
runner

,NAME,mean,std,monte_carlo
0,A,13.11,0.15,"[13.176184123032755, 13.060369477215886, 13.47..."
1,B,13.17,0.15,"[13.2514153541655, 13.230136844466045, 13.2779..."
2,C,12.99,0.17,"[13.053477246621568, 13.381068771608335, 12.63..."
3,D,12.96,0.20,"[12.54210336477075, 13.21492539692168, 12.4998..."
4,E,13.25,0.14,"[13.033248793320244, 13.281981203167181, 13.18..."
5,F,13.00,0.16,"[12.97901422717844, 13.01591314138514, 12.9421..."
6,G,13.40,0.17,"[13.48996718876471, 13.590274163503606, 13.382..."
7,H,13.29,0.20,"[13.249374380804586, 13.393153926554637, 12.99..."


Once we simulated the data, we can get the probability of each runner to win.

In [18]:
# Probability to finish in top x positions
top_x = 1
tmp_probs = pd.DataFrame((pd.DataFrame(list(runner['monte_carlo']),index=runner.NAME).rank()<=top_x).sum(axis=1)/sims)
tmp_probs.reset_index(inplace=True)
tmp_probs.columns=['NAME', 'Probability']
tmp_probs

,NAME,Probability
0,A,0.071
1,B,0.040
2,C,0.262
3,D,0.348
4,E,0.016
5,F,0.237
6,G,0.004
7,H,0.022


Similarly, we can estimate the probability of each runner to be on the podium, i.e. in the top 3 positions.

In [19]:
# Probability to finish in top x positions 
top_x = 3 # in top three positions
tmp_probs = pd.DataFrame((pd.DataFrame(list(runner['monte_carlo']),index=runner.NAME).rank()<=top_x).sum(axis=1)/sims)
tmp_probs.reset_index(inplace=True)
tmp_probs.columns=['NAME', 'Probability']
tmp_probs

,NAME,Probability
0,A,0.390
1,B,0.242
2,C,0.699
3,D,0.693
4,E,0.120
5,F,0.688
6,G,0.033
7,H,0.135
